In [10]:
#Labeling the genre to number and vice versa
def label2int(ch):
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    return chr(i)

In [16]:
import os
import glob
import pandas as pd
import math

import numpy as np
from scipy.misc import imread, imsave, imresize
from skimage.feature import canny
from skimage.morphology import reconstruction
from scipy.ndimage import gaussian_filter
from skimage.filters import threshold_otsu
from natsort import natsorted
import matplotlib.pyplot as plt
%matplotlib inline

# Path for data
path = os.getcwd()

# Input image dimensions
img_rows, img_cols = 40, 25

# Keep or not the initial image aspect ratio
keepRatio = False

# Suffix of the created directories and files
suffix = "Preproc_" + str(img_rows) + "_"+ str(img_cols) + ("_kr" if keepRatio else "")

# Create the directories if needed
if not os.path.exists(os.getcwd()+"/data/train"+suffix ):
    os.makedirs(os.getcwd()+"/data/train"+suffix)
if not os.path.exists( os.getcwd()+"/data/test"+suffix ):
    os.makedirs(os.getcwd()+"/data/test"+suffix)
    
    
### Images preprocessing ###

for setType in ["train", "test"]:
    files = natsorted(glob.glob(os.getcwd()+"/data/"+setType+"/*"))
    data = np.zeros((len(files), img_rows, img_cols)) 
    
    for i, filepath in enumerate(files):
        image = imread(filepath, True) #True: grayscale

        if keepRatio:
            maxSize = max(image.shape[0], image.shape[1])
            
            # Size of the resized image, keeping aspect ratio
            imageWidth = math.floor(img_rows*image.shape[0]/maxSize)
            imageHeigh = math.floor(img_cols*image.shape[1]/maxSize)
            
            # Compute deltas to center image 
            dRows = (img_rows-imageWidth)//2
            dCols = (img_cols-imageHeigh)//2
                        
            imageResized = np.zeros((img_rows, img_cols))
            imageResized[dRows:dRows+imageWidth, dCols:dCols+imageHeigh] = imresize(image, (imageWidth, imageHeigh))
            
            # Fill the empty image with the median value of the border pixels
            val = np.median(np.append(imageResized[dRows,:],
                                      (imageResized[dRows+imageWidth-1,:],
                                      imageResized[:,dCols],
                                      imageResized[:,dCols+imageHeigh-1])))
                                      
            # If rows were left blank
            if(dRows>0):
                imageResized[0:dRows,:].fill(val)
                imageResized[dRows+imageWidth:,:].fill(val)
                
            # If columns were left blank
            if(dCols>0):
                imageResized[:,0:dCols].fill(val)
                imageResized[:,dCols+imageHeigh:].fill(val)
        else:
            imageResized = imresize(image, (img_rows, img_cols))
            
        data[i] = imageResized
        
    data = data[:,np.newaxis,:,:] 
    
    # Makes values floats between 0 and 1
    data = data.astype('float32')
    data /= 255
    
    # Save the data as numpy file
    np.save(os.getcwd()+"/data/"+setType+suffix+".npy", data)

    
### Labels preprocessing ###

# Load labels
y_train = pd.read_csv(os.getcwd()+"/data/trainLabels.csv").values[:,1] #Keep only label

# Convert labels to one-hot vector
Y_train = np.zeros((y_train.shape[0], len(np.unique(y_train))))

for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # One-hot

# Save preprocessed label to numpy file
np.save(os.getcwd()+"/data/"+"labelsPreproc.npy", Y_train)

In [17]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split


batch_size = 200
nb_classes = 62 # A-Z, a-z and 0-9
nb_epoch = 500

# Input image dimensions
img_rows, img_cols = 40, 25

# Path of data files
path = os.getcwd()

# Load the preprocessed data and labels
X_train_all = np.load(os.getcwd()+"/data/"+"/trainPreproc_"+str(img_rows)+"_"+str(img_cols)+".npy")
Y_train_all = np.load(os.getcwd()+"/data/"+"/labelsPreproc.npy")

# Do multiple learnings and predictions with the aim of averaging them
for runID in range (18):    
    # Split in train and validation sets to get the "best" model.
    X_train, X_val, Y_train, Y_val = \
        train_test_split(X_train_all, Y_train_all, test_size=0.25, stratify=np.argmax(Y_train_all, axis=1))
   
                             
    # Parametrize the image augmentation class
    datagen = ImageDataGenerator(
        rotation_range = 20,
        width_shift_range = 0.15,
        height_shift_range = 0.15,
        shear_range = 0.4,
        zoom_range = 0.3,                    
        channel_shift_range = 0.1,
        channel_flip = True, 
        channel_flip_max = 1.) 

    ### CNN MODEL ###
    model = Sequential()

    model.add(Convolution2D(128, 3, 3, border_mode='same', init='he_normal', activation = 'relu', input_shape=(1,img_rows, img_cols)))
    model.add(Convolution2D(128, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(256, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(256, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(4096, init='he_normal', activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, init='he_normal', activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, init='he_normal', activation = 'softmax'))

    ### LEARNING ###
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adadelta',  
                  metrics=["accuracy"])
            
    saveBestModel = ModelCheckpoint("best.kerasModelWeights", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True)

    # Make the model learn using the image generator
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        samples_per_epoch=len(X_train),
                        nb_epoch=nb_epoch, 
                        validation_data=(X_val, Y_val),
                        callbacks=[saveBestModel],
                        verbose=1)

    ### PREDICTION ###
                        
    # Load the model with the highest validation accuracy
    model.load_weights("best.kerasModelWeights")

    # Load Kaggle test set
    X_test = np.load(os.getcwd()+"/data/testPreproc_"+str(img_rows)+"_"+str(img_cols)+".npy")

    # Predict the class (give the index in the one-hot vector of the most probable class)
    Y_test_pred = model.predict_classes(X_test)
    
    # Translate integers to character labels
    vInt2label = np.vectorize(int2label)
    Y_test_pred = vInt2label(Y_test_pred)
    
    # Save the predicitions in Kaggle format
    np.savetxt(os.getcwd()+"/data/CNN_pred_"+str(runID)+".csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')

Epoch 1/500
4712/4712 [==============================] - 294s - loss: 4.0525 - acc: 0.0514 - val_loss: 3.8454 - val_acc: 0.0433
Epoch 2/500
4712/4712 [==============================] - 300s - loss: 3.8559 - acc: 0.0649 - val_loss: 3.8705 - val_acc: 0.0834
Epoch 3/500
4712/4712 [==============================] - 297s - loss: 3.7805 - acc: 0.0821 - val_loss: 3.6091 - val_acc: 0.0993
Epoch 4/500
4712/4712 [==============================] - 290s - loss: 3.8040 - acc: 0.0876 - val_loss: 3.7417 - val_acc: 0.1273
Epoch 5/500
4712/4712 [==============================] - 293s - loss: 3.6721 - acc: 0.1231 - val_loss: 3.2340 - val_acc: 0.2502
Epoch 6/500
4712/4712 [==============================] - 300s - loss: 3.4452 - acc: 0.1640 - val_loss: 3.3742 - val_acc: 0.1712
Epoch 7/500
4712/4712 [==============================] - 296s - loss: 3.3350 - acc: 0.1876 - val_loss: 2.4486 - val_acc: 0.3985
Epoch 8/500
4712/4712 [==============================] - 302s - loss: 3.3343 - acc: 0.2020 - val_loss: 2

KeyboardInterrupt: 

In [20]:
model.load_weights("best.kerasModelWeights")

    # Load Kaggle test set
X_test = np.load(os.getcwd()+"/data/testPreproc_"+str(img_rows)+"_"+str(img_cols)+".npy")

    # Predict the class (give the index in the one-hot vector of the most probable class)
Y_test_pred = model.predict_classes(X_test)
    
    # Translate integers to character labels
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(Y_test_pred)
    
    # Save the predicitions in Kaggle format
np.savetxt(os.getcwd()+"/data/CNN_new4025.csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')

6220/6220 [==============================] - 127s   


In [19]:
# List of prediction files in path
allPredsFiles = glob.glob(os.getcwd()+"/data/CNN_pred_*.csv")

allPreds = []

for file in allPredsFiles:
    preds = pd.read_csv(file).values
    predictionTemplate = preds 
    allPreds.append(preds[:,1:2]) 

# Stacks all the predictions 
predsStacked = np.hstack(allPreds) 

# Create the averaged result array, copying the template
predsAveraged = np.array(predictionTemplate)

# For each prediction, get the most frequent one
for i in range(predsStacked.shape[0]):
    (values,counts) = np.unique(predsStacked[i], return_counts=True) 
    ind=np.argmax(counts) 
    predsAveraged[i,1] = values[ind]  # gets the most frequent label
    
# Save the averaged predicitions in csv file
np.savetxt(os.getcwd()+"/data/avg_pred.csv", np.c_[predsAveraged], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')